In [61]:
import os
from pathlib import Path

import torch
from transformers import OpenAIGPTLMHeadModel, OpenAIGPTTokenizer, pipeline, set_seed

In [43]:
model = "openai-gpt"
cache_dir = Path("./models")

In [44]:
tokenizer = OpenAIGPTTokenizer.from_pretrained(
    pretrained_model_name_or_path=model, cache_dir=cache_dir
)
model = OpenAIGPTLMHeadModel.from_pretrained(
    pretrained_model_name_or_path=model, cache_dir=cache_dir
)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/816k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/458k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/656 [00:00<?, ?B/s]

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


model.safetensors:   0%|          | 0.00/479M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

In [29]:
inputs = tokenizer("black shoes for", return_tensors="pt")
outputs = model(**inputs)

In [31]:
last_hidden_states = outputs.last_hidden_state
last_hidden_states.shape

torch.Size([1, 3, 768])

In [46]:
model_local_path = Path(
    "./models/models--openai-gpt/snapshots/1e0d4f3028acbffb47fe933cea64619c5ec1a002"
)

In [56]:
generator = pipeline(
    task="text-generation", tokenizer=str(model_local_path), model=str(model_local_path)
)

In [62]:
%%timeit
generator("black running", max_length=5, num_return_sequences=5, truncation=True)

90 ms ± 2.38 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [88]:
generator("<cars> a red ", max_length=10, num_return_sequences=5, truncation=True)

[{'generated_text': '<cars> a red  porsche 306, and'},
 {'generated_text': '<cars> a red  toyota was parked in her'},
 {'generated_text': '<cars> a red  pontiac. the last'},
 {'generated_text': '<cars> a red  honda accord pulled up,'},
 {'generated_text': '<cars> a red  mazda and a white'}]